In [1]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard


In [2]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier_model = mobilenet_v2


In [3]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])


Metal device set to: Apple M1


2022-08-08 05:06:31.988691: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-08 05:06:31.988790: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
batch_size = 32
img_height = 224
img_width = 224
data_root = r"TB_dataset"


train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 662 files belonging to 2 classes.
Using 596 files for training.
Found 662 files belonging to 2 classes.
Using 66 files for validation.


In [5]:
class_names = np.array(train_ds.class_names)
print(class_names)

['NORMAL' 'TB']


In [6]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

In [7]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

2022-08-08 05:07:49.120764: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(32, 224, 224, 3)
(32,)


2022-08-08 05:07:52.925077: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [9]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2

In [10]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [11]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

2022-08-08 05:08:18.932551: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


(32, 1280)


In [12]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:
predictions = model(image_batch)

In [14]:
predictions.shape

TensorShape([32, 2])

In [15]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [16]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/10


2022-08-08 05:09:13.948247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - ETA: 0s - loss: 0.6120 - acc: 0.6745

2022-08-08 05:09:34.030667: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 26s 740ms/step - loss: 0.6120 - acc: 0.6745 - val_loss: 0.5144 - val_acc: 0.7424
Epoch 2/10
19/19 [==============================] - 11s 511ms/step - loss: 0.4527 - acc: 0.7987 - val_loss: 0.4593 - val_acc: 0.7727
Epoch 3/10
19/19 [==============================] - 9s 485ms/step - loss: 0.4078 - acc: 0.8289 - val_loss: 0.4342 - val_acc: 0.7879
Epoch 4/10
19/19 [==============================] - 9s 473ms/step - loss: 0.3786 - acc: 0.8406 - val_loss: 0.4165 - val_acc: 0.7879
Epoch 5/10
19/19 [==============================] - 10s 539ms/step - loss: 0.3566 - acc: 0.8523 - val_loss: 0.4031 - val_acc: 0.8485
Epoch 6/10
19/19 [==============================] - 9s 455ms/step - loss: 0.3391 - acc: 0.8557 - val_loss: 0.3925 - val_acc: 0.8636
Epoch 7/10
19/19 [==============================] - 9s 459ms/step - loss: 0.3245 - acc: 0.8691 - val_loss: 0.3840 - val_acc: 0.8636
Epoch 8/10
19/19 [==============================] - 9s 467ms/step - loss: 0.3120 - a

In [17]:
t = time.time()

export_path = r"models/basic_tb/{}".format(int(t))
model.save(export_path)
model.save('model_{}.h5'.format(int(t)))

export_path

INFO:tensorflow:Assets written to: models/basic_tb/1659957142/assets


INFO:tensorflow:Assets written to: models/basic_tb/1659957142/assets


'models/basic_tb/1659957142'

In [20]:
import gradio as gr
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [21]:
model = keras.models.load_model('models/basic_tb/1659957142')


In [22]:
class_names = ['NORMAL','TUBERCULOSIS']
def predict_image(img):
  img_4d=img.reshape((-1,224,224,3))
    
  prediction=model.predict(img_4d).flatten()
  return {class_names[i]: float(prediction[i]) for i in range(2)}

In [ ]:
image = gr.inputs.Image(shape=(224,224))
label = gr.outputs.Label(num_top_classes=2)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')

/Users/pips/Desktop/tf_files/lunghealth_tf/env/lib/python3.8/site-packages/gradio/inputs.py:256: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/pips/Desktop/tf_files/lunghealth_tf/env/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/pips/Desktop/tf_files/lunghealth_tf/env/lib/python3.8/site-packages/gradio/outputs.py:196: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/Users/pips/Desktop/tf_files/lunghealth_tf/env/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


2022-08-08 05:14:18.300981: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 76ms/step
